### Read the script

In [ ]:
import os
import numpy as np
import mne
from ieeg.io import get_data, raw_from_layout, save_derivative
from ieeg.navigate import trial_ieeg, outliers_to_nan, channel_outlier_marker, crop_empty_data
from ieeg.calc.scaling import rescale
from ieeg.viz.ensemble import chan_grid
from ieeg.timefreq.utils import crop_pad, wavelet_scaleogram
from ieeg.viz.parula import parula_map
save_dir='C:\\Users\\bl314\\Box\\CoganLab\\IndividualMeetings\\Baishen\\ieeg_results\\lexical_delay'

In [ ]:
HOME = os.path.expanduser("~")
LAB_root = os.path.join(HOME, "Box", "CoganLab")
layout = get_data("LexicalDecRepDelay", root=LAB_root)
subjects = layout.get(return_type="id", target="subject")
subject = subjects[0]
subject_Tag = 'D103'

#### Plot subj

In [ ]:
# plot subject
# !!!!! problems: cannot find the surface file##########
from ieeg.viz.mri import plot_subj, plot_on_average

# plot the subject brain
fig1 = plot_subj(subject_Tag)
#C:\Users\bl314\Box\ECoG_Recon\D103
#Label every electrode
mne.viz.set_3d_view(fig1, azimuth=150, elevation=70, focalpoint="auto",
                    distance="auto")

#### Load subj

In [ ]:
raw = raw_from_layout(layout.derivatives['derivatives/clean'], subject=subject, desc='clean',extension='.edf',preload=True)
# Choose preload=False to release RAM space (but )
raw = crop_empty_data(raw)

### Remove bad channels
https://ieeg-pipelines.readthedocs.io/en/latest/auto_examples/plot_spectrograms.html


In [ ]:
# Currently we don't do EEG channel removing.

# Remove EEG channels for D101
# eeg_channels_to_exclude = ['Fp1', 'Fp2', 'F3','F4','C3','C4','P3','P4','F7','F8','T3','T4','T5','T6','O1','O2','Fz','Cz','Pz']

# Remove EEG channels for D103 (same)
#eeg_channels_to_exclude (bad) = ['F7', 'F8', 'T5', 'T6', 'F3', 'F4', 'C3', 'C4', 'P3', 'P4',
#                            'T3', 'T4', 'O1', 'O2']
eeg_channels_to_exclude = ['FZ', 'CZ', 'PZ', 'F7', 'F8', 'T5', 'T6', 'FP1', 'FP2', 'F3', 'F4', 'C3', 'C4', 'P3', 'P4', 'O1', 'O2', 'T3', 'T4']
# Problems: !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!Channel(s) Fz, Cz, Pz, Fp1, Fp2 not found, nothing dropped. But these channels exist in the sub-D0103_task-LexicalDecRepDelay_channels.tsv!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

# Remove EEG channels for D107

raw.drop_channels(eeg_channels_to_exclude)
raw.load_data()

In [ ]:
# mark channel outliers as bad
raw.info['bads'] = channel_outlier_marker(raw, 3, 2)
# Exclude bad channels
raw.drop_channels(raw.info['bads'])
# Currently we don't do EEG channel removing.

# Remove EEG channels for D101
eeg_channels_to_exclude = ['Fp1', 'Fp2', 'F3', 'F4', 'C3', 'C4', 'P3', 'P4', 'F7', 'F8', 'T3', 'T4', 'T5', 'T6', 'O1',
                           'O2', 'Fz', 'Cz', 'Pz']
# Remove EEG channels for D103 (same)
# eeg_channels_to_exclude = ['F7', 'F8', 'T5', 'T6', 'F3', 'F4', 'C3', 'C4', 'P3', 'P4',
#                            'T3', 'T4', 'O1', 'O2']

# Problems: !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!Channel(s) Fz, Cz, Pz, Fp1, Fp2 not found, nothing dropped. But these channels exist in the sub-D0103_task-LexicalDecRepDelay_channels.tsv!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# Remove EEG channels for D107

raw.drop_channels(eeg_channels_to_exclude)
raw.load_data()

In [ ]:
# Detect and remove muscle artifact channels
# Also plot the subject brain
for epoch, t in zip(
        ("Auditory_stim","Resp"),
        ((-0.5, 1.5), (-0.5, 1)),
      ):
    t1 = t[0] - 0.5
    t2 = t[1] + 0.5
    times = (t1, t2)
    trials = trial_ieeg(raw, epoch, times, preload=True)
    outliers_to_nan(trials, outliers=10)
    spectra_wavelet = wavelet_scaleogram(trials, n_jobs=-3, decim=int(
        raw.info['sfreq'] / 200)) # 1/10 of the timepionts, don't take too long
    crop_pad(spectra_wavelet, "0.5s") # cut the first and final 0.5s, change to zero
 
    if not os.path.exists(os.path.join(save_dir, subject)):
        os.mkdir(os.path.join(save_dir, subject))
    # if epoch=='Auditory_stim/CORRECT':
    #     filename = os.path.join(save_dir, subject,'Auditory_stim-CORRECT-wavelet-tfr.h5')
    # elif epoch=='Resp/CORRECT':
    #     filename = os.path.join(save_dir, subject,'Resp-CORRECT-wavelet-tfr.h5')
    filename = os.path.join(save_dir, subject,f'{epoch}-wavelet-tfr.h5')
    mne.time_frequency.write_tfrs(filename, spectra_wavelet, overwrite=True)
    del trials, spectra_wavelet

In [ ]:
# Then check the auditory_stim
filename = os.path.join(save_dir, subject,'Auditory_stim-wavelet-tfr.h5')
spectra_auditory = mne.time_frequency.read_tfrs(filename)

# crop the baseline
base_wavelet = spectra_auditory.copy().crop(-0.5, 0)
# Average, and do baseline correction
base_wavelet = base_wavelet.average(lambda x: np.nanmean(x, axis=0), copy=True)
    
spectra_auditory = spectra_auditory.average(lambda x: np.nanmean(x, axis=0), copy=True)   
spectra_auditory_blc = rescale(spectra_auditory, base_wavelet, copy=True, mode='ratio')
spectra_auditory_blc._data = np.log10(spectra_auditory_blc._data) * 20

chan_grid(spectra_auditory_blc, size = (20,10),vlim=(-2, 2), cmap=parula_map)

In [ ]:
# Check the wavelet spectrograms
# Check the motor signal first
filename = os.path.join(save_dir, subject,'Resp-wavelet-tfr.h5')
spectra_motor = mne.time_frequency.read_tfrs(filename)

spectra_motor = spectra_motor.average(lambda x: np.nanmean(x, axis=0), copy=True)   
spectra_motor_blc = rescale(spectra_motor, base_wavelet, copy=True, mode='ratio')
spectra_motor_blc._data = np.log10(spectra_motor_blc._data) * 20

chan_grid(spectra_motor_blc, size = (20,10),vlim=(-2, 2), cmap=parula_map)

In [ ]:
# Manually mark muscle bad channels

# D101: 
#muscle_artifacts_chn=['RTMM11','RTMM12','RTAM13','RTAM14','RTAM15']

# D103:
muscle_artifacts_chn=['LTMM1']


# D107, no muscle channels found
if muscle_artifacts_chn:
    raw.drop_channels(muscle_artifacts_chn)
    raw.load_data()

In [ ]:
# CAR (common average reference) DONT DO THIS NOW
# raw.set_eeg_reference(ref_channels="average", ch_type='seeg')

In [ ]:
# Check if derivatives folder exists and create if not
bids_root='C:\\Users\\bl314\\Box\\CoganLab\\BIDS-1.0_LexicalDecRepDelay\\BIDS'
if not os.path.exists(os.path.join(bids_root, "derivatives")):
    os.mkdir(os.path.join(bids_root, "derivatives"))
    os.mkdir(os.path.join(bids_root, "derivatives", "clean bchrm"))
elif not os.path.exists(os.path.join(bids_root, "derivatives", "clean bchrm")):
    os.mkdir(os.path.join(bids_root, "derivatives", "clean bchrm"))
save_derivative(raw, layout, "clean bchrm", True)

### Remove bad trials and plot spectrogram
https://ieeg-pipelines.readthedocs.io/en/latest/auto_examples/plot_spectrograms_wavelet.html

In [ ]:
# Epoches: https://mne.tools/stable/generated/mne.Epochs.html#mne.Epochs.__getitem__
for epoch, t in zip(
        ("Auditory_stim/CORRECT", "Delay/CORRECT", "Resp/CORRECT"),
        ((-0.5, 1.5), (-0.5, 1.5), (-0.5, 1)),
      ):
    
    # Add time point for timefrequency and crop it
    t1 = t[0] - 0.5
    t2 = t[1] + 0.5
    times = (t1, t2)
    trials = trial_ieeg(raw, epoch, times, preload=True)
    # eeg file is big. If try load multiple subjects. 
    # pointing
    
    # remove bad channels
    outliers_to_nan(trials, outliers=10)
    #Learned from Aaron's sentence rep
    #https://github.com/coganlab/SentenceRep_analysis/blob/main/analysis/check/multitaper_spec.py
    
    # Multitaper wavelet timefreq
    #freq = np.arange(10, 200., 6.)
    #spectra = spectrogram(raw, freq, 'Auditory_stim', -1.2, 1.2, 'Cue', -0.5, 0,
    #                      n_jobs=-3, verbose=10, time_bandwidth=10, n_cycles=freq/2)
    # verbose: extra messages
    # https://mne.tools/dev/generated/mne.verbose.html#mne.verbose
    #spectra = spectrogram(trials, baseline=baselines, freqs=freq, n_jobs=-3, time_bandwidth=10, n_cycles=freq/2)
    # https://mne.tools/dev/generated/mne.time_frequency.tfr_array_multitaper.html (Notes)
    #crop_pad(spectra, "0.5s")
    freq = np.linspace(0.5, 200, num=80)
    kwargs = dict(average=False, n_jobs=-3, freqs=freq, return_itc=False,
                  n_cycles=freq/2, time_bandwidth=4,
                  # n_fft=int(trials.info['sfreq'] * 2.75),
                  decim=20, )
                  # adaptive=True)
    spectra = trials.compute_tfr(method="multitaper",  **kwargs)
    crop_pad(spectra, "0.5s") # cut the first and final 0.5s, change to zero
    
    # crop the baseline
    if epoch == "Auditory_stim":
        base = spectra.copy().crop(-0.5, 0)
            # Average, and do baseline correction
        base = base.average(lambda x: np.nanmean(x, axis=0), copy=True)
        
    spectra = spectra.average(lambda x: np.nanmean(x, axis=0), copy=True)
    rescale(spectra._data, base._data, mode='ratio', axis=-1)
    
    # Save spectrograms
    #fnames = [os.path.relpath(f, layout.root) for f in good.filenames]
    #spectra.info['subject_info']['files'] = tuple(fnames)
    spectra.info['bads'] = raw.info['bads']
    #https://github.com/coganlab/SentenceRep_analysis/blob/main/analysis/check/multitaper_spec.py
    if not os.path.exists(os.path.join(save_dir, subject)):
        os.mkdir(os.path.join(save_dir, subject))
    filename = os.path.join(save_dir, subject,f'{epoch}-tfr.h5')
    mne.time_frequency.write_tfrs(filename, spectra, overwrite=True)
    #spectra.save(os.path.join(save_dir,subject, 'Auditor-avg.fif'))
    

### Multitaper spectrogram plot

#### Plot data

In [ ]:
# del spectra
epoch =  "CORRECT"#"Delay"# "Auditory_stim"#"CORRECT"#,#
filename = os.path.join(save_dir, subject,f'{epoch}-tfr.h5')
spectra = mne.time_frequency.read_tfrs(filename)
chan_grid(spectra, size = (20,10),vlim=(0.7, 1.4), fmax=200, show=True, cmap=parula_map,)# yscale="log")